## 1. Setup

Import the resolver and graph components.

In [1]:
import sys
sys.path.insert(0, '/home/stellar/PublicRepo/YMHY/anime_streamer_2_0')

from pathlib import Path
from cores.flow_core.tokenizer import Tokenizer
from cores.flow_core.parser import parse
from cores.flow_core.resolver import resolve_with_graph, Resolver
from cores.flow_core.models import DependencyGraph, EdgeType

print("✓ Flow Core components imported successfully")
print(f"\nEdge types available:")
print(f"  BACKWARD_REF: {EdgeType.BACKWARD_REF!r}")
print(f"  FORWARD_REF: {EdgeType.FORWARD_REF!r}")
print(f"  SLOT: {EdgeType.SLOT!r}")
print(f"  IMPORT: {EdgeType.IMPORT!r}")
print(f"  CONTEXT_REF: {EdgeType.CONTEXT_REF!r}")

✓ Flow Core components imported successfully

Edge types available:
  BACKWARD_REF: 'backward'
  FORWARD_REF: 'forward'
  SLOT: 'slot'
  IMPORT: 'import'
  CONTEXT_REF: 'context_ref'


In [2]:
tokenizer = Tokenizer()
print("✓ Tokenizer instance created successfully")

✓ Tokenizer instance created successfully


## 2. Building a Dependency Graph

A simple example showing how to build a graph from source code.

In [3]:
source = '''
@greeting |<<<Hello, World!>>>|.
@farewell |<<<Goodbye!>>>|.
@main |$greeting|<<<...>>>|$farewell|.
@out |$main|.
'''

# Pipeline: tokenize -> parse -> resolve with graph
tokens = tokenizer.tokenize(source)
flow_file = parse(tokens)
resolved, graph = resolve_with_graph(flow_file)

print("DependencyGraph:")
print(f"  Nodes: {sorted(graph.nodes)}")
print(f"  Edges: {len(graph.edges)}")
for edge in sorted(graph.edges):
    print(f"    {edge[0]} --[{edge[2]}]--> {edge[1]}")
print(f"  Files: {list(graph.files)}")
print(f"  File Refs: {list(graph.file_refs)}")

DependencyGraph:
  Nodes: ['farewell', 'greeting', 'main', 'out']
  Edges: 3
    main --[backward]--> farewell
    main --[backward]--> greeting
    out --[backward]--> main
  Files: []
  File Refs: []


## 3. Graph Query Methods

The DependencyGraph provides methods to query the graph structure.

In [4]:
# What nodes does 'main' depend on?
print(f"Dependencies of 'main': {graph.dependencies('main')}")

# What nodes depend on 'greeting'?
print(f"Dependents of 'greeting': {graph.dependents('greeting')}")

# Get edges from/to a specific node
print(f"\nEdges from 'main': {graph.edges_from('main')}")
print(f"Edges to 'main': {graph.edges_to('main')}")

Dependencies of 'main': {'farewell', 'greeting'}
Dependents of 'greeting': {'main'}

Edges from 'main': {('main', 'greeting', 'backward'), ('main', 'farewell', 'backward')}
Edges to 'main': {('out', 'main', 'backward')}


## 4. Export Formats

The graph can be exported to DOT (Graphviz), Mermaid, and JSON formats.

### 4.1 DOT Format (Graphviz)

In [5]:
dot_output = graph.to_dot()
print(dot_output)

digraph FlowDependencies {
    rankdir=TB;
    node [shape=box];

    "farewell";
    "greeting";
    "main";
    "out";

    "main" -> "farewell" [label="backward" color="blue"];
    "main" -> "greeting" [label="backward" color="blue"];
    "out" -> "main" [label="backward" color="blue"];
}


In [6]:
# To render with Graphviz (if installed), you can save and process:
# dot_path = Path('/tmp/flow_graph.dot')
# dot_path.write_text(dot_output)
# Then run: dot -Tpng /tmp/flow_graph.dot -o /tmp/flow_graph.png

# Or use graphviz Python package if available:
try:
    import graphviz
    g = graphviz.Source(dot_output)
    display(g)
except ImportError:
    print("graphviz package not installed. Run: pip install graphviz")
    print("DOT output saved above can be rendered with Graphviz CLI.")

graphviz package not installed. Run: pip install graphviz
DOT output saved above can be rendered with Graphviz CLI.


### 4.2 Mermaid Format

In [7]:
mermaid_output = graph.to_mermaid()
print(mermaid_output)

flowchart TB
    farewell["farewell"]
    greeting["greeting"]
    main["main"]
    out["out"]
    main -->|backward| farewell
    main -->|backward| greeting
    out -->|backward| main


In [8]:
# Mermaid can be rendered in markdown or with mermaid.js
# You can paste the output into any Mermaid-compatible editor
# or use the Mermaid Live Editor: https://mermaid.live/

from IPython.display import display, HTML, Markdown

# Display as markdown (may not render in all environments)
display(Markdown(f"```mermaid\n{mermaid_output}\n```"))

```mermaid
flowchart TB
    farewell["farewell"]
    greeting["greeting"]
    main["main"]
    out["out"]
    main -->|backward| farewell
    main -->|backward| greeting
    out -->|backward| main
```

### 4.3 JSON Format

In [9]:
import json

json_output = graph.to_json()
print(json_output)

{
  "nodes": [
    "farewell",
    "greeting",
    "main",
    "out"
  ],
  "edges": [
    {
      "from": "main",
      "to": "farewell",
      "type": "backward"
    },
    {
      "from": "main",
      "to": "greeting",
      "type": "backward"
    },
    {
      "from": "out",
      "to": "main",
      "type": "backward"
    }
  ],
  "files": [],
  "file_refs": []
}


In [10]:
# The JSON format is ideal for programmatic access
data = json.loads(json_output)
print(f"\nParsed JSON:")
print(f"  Node count: {len(data['nodes'])}")
print(f"  Edge count: {len(data['edges'])}")
print(f"  Edge types: {set(e['type'] for e in data['edges'])}")


Parsed JSON:
  Node count: 4
  Edge count: 3
  Edge types: {'backward'}


## 5. Complex Graph Examples

Let's explore graphs with different edge types.

### 5.1 Forward References

In [11]:
source = '''
@intro |<<<Coming up: >>>|^conclusion|.
@middle |<<<The middle part>>>|.
@conclusion |<<<The End>>>|.
@out |$intro|$middle|$conclusion|.
'''

tokens = tokenizer.tokenize(source)
flow_file = parse(tokens)
resolved, graph = resolve_with_graph(flow_file)

print("Forward reference example:")
print(f"  Edges:")
for edge in sorted(graph.edges):
    style = "(forward)" if edge[2] == EdgeType.FORWARD_REF else ""
    print(f"    {edge[0]} --[{edge[2]}]--> {edge[1]} {style}")

print(f"\n{graph.to_mermaid()}")

Forward reference example:
  Edges:
    intro --[forward]--> conclusion (forward)
    out --[backward]--> conclusion 
    out --[backward]--> intro 
    out --[backward]--> middle 

flowchart TB
    conclusion["conclusion"]
    intro["intro"]
    middle["middle"]
    out["out"]
    intro -.->|forward| conclusion
    out -->|backward| conclusion
    out -->|backward| intro
    out -->|backward| middle


### 5.2 File References (context_ref edges)

In [12]:
source = '''
@docs |<<<Documentation: >>>|++./README.md|.
@config |<<<Config: >>>|++./config.yaml|.
@combined |$docs|$config|.
@out |$combined|.
'''

tokens = tokenizer.tokenize(source)
flow_file = parse(tokens)
resolved, graph = resolve_with_graph(flow_file)

print("File reference example:")
print(f"  All edges:")
for edge in sorted(graph.edges):
    print(f"    {edge[0]} --[{edge[2]}]--> {edge[1]}")

print(f"\n  File refs (computed property): {list(graph.file_refs)}")

File reference example:
  All edges:
    combined --[backward]--> config
    combined --[backward]--> docs
    config --[context_ref]--> ./config.yaml
    docs --[context_ref]--> ./README.md
    out --[backward]--> combined

  File refs (computed property): [PosixPath('config.yaml'), PosixPath('README.md')]


### 5.3 Nested Nodes with Slots

In [13]:
source = '''
@document
|@header |<<<Header>>>|.
|@body |<<<Body>>>|.
|.

@custom_header |<<<Custom Header Content>>>|.
$document.header = $custom_header

@out |$document|.
'''

tokens = tokenizer.tokenize(source)
flow_file = parse(tokens)
resolved, graph = resolve_with_graph(flow_file)

print("Slot reference example:")
print(f"  Edges:")
for edge in sorted(graph.edges):
    edge_style = "(slot)" if edge[2] == EdgeType.SLOT else ""
    print(f"    {edge[0]} --[{edge[2]}]--> {edge[1]} {edge_style}")

Slot reference example:
  Edges:
    out --[backward]--> document 


## 6. Graph Merging

Multiple graphs can be merged for cross-file analysis.

In [14]:
# Create two separate graphs
source1 = '''
@helper |<<<Helper function>>>|.
@util |$helper|.
@out |$util|.
'''

source2 = '''
@main |<<<Main logic>>>|.
@app |$main|.
@out |$app|.
'''

tokens1 = tokenizer.tokenize(source1)
flow_file1 = parse(tokens1)
_, graph1 = resolve_with_graph(flow_file1)

tokens2 = tokenizer.tokenize(source2)
flow_file2 = parse(tokens2)
_, graph2 = resolve_with_graph(flow_file2)

# Merge the graphs
merged = DependencyGraph.merge(graph1, graph2)

print(f"Graph 1 nodes: {sorted(graph1.nodes)}")
print(f"Graph 2 nodes: {sorted(graph2.nodes)}")
print(f"Merged nodes: {sorted(merged.nodes)}")
print(f"Merged edges: {len(merged.edges)}")

Graph 1 nodes: ['helper', 'out', 'util']
Graph 2 nodes: ['app', 'main', 'out']
Merged nodes: ['app', 'helper', 'main', 'out', 'util']
Merged edges: 4


## 7. Empty Graph

In [15]:
# Create an empty graph
empty = DependencyGraph.empty()

print(f"Empty graph:")
print(f"  Nodes: {empty.nodes}")
print(f"  Edges: {empty.edges}")
print(f"  Files: {empty.files}")
print(f"  File refs: {empty.file_refs}")

Empty graph:
  Nodes: frozenset()
  Edges: frozenset()
  Files: frozenset()
  File refs: frozenset()


## 8. Parse Sample Files

Let's build graphs from the sample `.flow` files.

In [16]:
samples_dir = Path('./playground/samples')
if samples_dir.exists():
    for sample_file in sorted(samples_dir.glob('*.flow')):
        print(f"\n{'=' * 60}")
        print(f" {sample_file.name}")
        print(f"{'=' * 60}")
        
        source = sample_file.read_text()
        try:
            tokens = tokenizer.tokenize(source)
            flow_file = parse(tokens)
            resolved, graph = resolve_with_graph(
                flow_file, 
                base_path=samples_dir,
                source_path=str(sample_file)
            )
            
            print(f"Nodes: {sorted(graph.nodes)}")
            print(f"Edges ({len(graph.edges)}):")
            for edge in sorted(graph.edges):
                print(f"  {edge[0]} --[{edge[2]}]--> {edge[1]}")
            if graph.file_refs:
                print(f"File refs: {list(graph.file_refs)}")
        except Exception as e:
            print(f"Error: {e}")
else:
    print("Samples directory not found")

Samples directory not found


## 9. Interactive Graph Builder

Use this cell to experiment with your own Flow source code.

In [17]:
# Your FLOW source here
source = '''
@greeting |<<<Hello!>>>|.
@farewell |<<<Goodbye!>>>|.
@out |$greeting|$farewell|.
'''

try:
    tokens = tokenizer.tokenize(source)
    flow_file = parse(tokens)
    resolved, graph = resolve_with_graph(flow_file)
    
    print("=== Dependency Graph ===")
    print(f"Nodes: {sorted(graph.nodes)}")
    print(f"\nEdges:")
    for edge in sorted(graph.edges):
        print(f"  {edge[0]} --[{edge[2]}]--> {edge[1]}")
    
    print(f"\n=== Mermaid Export ===")
    print(graph.to_mermaid())
    
except Exception as e:
    print(f"Error: {e}")

=== Dependency Graph ===
Nodes: ['farewell', 'greeting', 'out']

Edges:
  out --[backward]--> farewell
  out --[backward]--> greeting

=== Mermaid Export ===
flowchart TB
    farewell["farewell"]
    greeting["greeting"]
    out["out"]
    out -->|backward| farewell
    out -->|backward| greeting
